<a href="https://colab.research.google.com/github/itstyren/inteaction-dynamics-in-social-dilemmas/blob/main/visualization/plot_result.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install wandb -qU

In [7]:
# Log in to your W&B account
import wandb
wandb.login()

True

In [12]:
api = wandb.Api()
run = api.run("tyren/Lattice/wkpp6bze")
run.history()

,robutness/average_defection_length,eval_interaction/average_interaction,payoff/defection_episode_payoff,payoff/cooperation_episode_payoff,_runtime,robutness/average_cooperation_length,results/coopereation_episode_rewards,eval_interaction/cooperation_interaction_ratio,train,rollout/avg_coop_for_defection,...,eval,train/prioritized_replay_beta,train/defection_reward,rollout/target_update,results/defection_episode_rewards,eval_payoff/episode_payoff,eval_payoff/cooperation_episode_payoff,eval_result/average_cooperation_length,results/termination_proportion,payoff/episode_payoff
0,NaN,NaN,NaN,NaN,16.160577,NaN,NaN,NaN,None,1.000000,...,None,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,16.196296,NaN,NaN,NaN,None,0.544133,...,None,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,16.221455,NaN,NaN,NaN,None,0.484418,...,None,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,16.250432,NaN,NaN,NaN,None,0.519813,...,None,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,16.274995,NaN,NaN,NaN,None,0.531667,...,None,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,16.300750,NaN,NaN,NaN,None,0.503901,...,None,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,16.325026,NaN,NaN,NaN,None,0.508070,...,None,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,16.349240,NaN,NaN,NaN,None,0.524065,...,None,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,16.372523,NaN,NaN,NaN,None,0.551178,...,None,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,16.423401,NaN,NaN,NaN,None,0.523443,...,None,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import os
os.environ["MUJOCO_GL"] = "egl"

import coax
import haiku as hk
import jax
import jax.numpy as jnp
import numpy as onp
import optax

from coax.utils import make_dmc


# the name of this script
name = 'sac'

# the dm_control MDP
env = make_dmc("walker", "walk")
env = coax.wrappers.TrainMonitor(env, name=name)


def func_pi(S, is_training):
    seq = hk.Sequential((
        hk.Linear(1024), hk.LayerNorm(-1, create_scale=True, create_offset=True), jax.nn.tanh,
        hk.Linear(1024), jax.nn.relu,
        hk.Linear(onp.prod(env.action_space.shape) * 2),
        hk.Reshape((*env.action_space.shape, 2)),
    ))
    x = seq(S)
    mu, logvar = x[..., 0], x[..., 1]
    return {'mu': mu, 'logvar': logvar}


def func_q(S, A, is_training):
    seq = hk.Sequential((
        hk.Linear(1024), hk.LayerNorm(-1, create_scale=True, create_offset=True), jax.nn.tanh,
        hk.Linear(1024), jax.nn.relu,
        hk.Linear(1), jnp.ravel
    ))
    X = jnp.concatenate((S, A), axis=-1)
    return seq(X)


# main function approximators
pi = coax.Policy(func_pi, env, proba_dist=coax.proba_dists.SquashedNormalDist(
    env.action_space,
    clip_logvar=(-10.0, 4.0),
))
q1 = coax.Q(func_q, env, action_preprocessor=pi.proba_dist.preprocess_variate)
q2 = coax.Q(func_q, env, action_preprocessor=pi.proba_dist.preprocess_variate)

# target network
q1_targ = q1.copy()
q2_targ = q2.copy()

# experience tracer
tracer = coax.reward_tracing.NStep(n=5, gamma=0.99, record_extra_info=True)
buffer = coax.experience_replay.SimpleReplayBuffer(capacity=1000000)
policy_regularizer = coax.regularizers.NStepEntropyRegularizer(pi,
                                                               beta=0.2,
                                                               gamma=tracer.gamma,
                                                               n=[tracer.n])

# updaters (use current pi to update the q-functions and use sampled action in contrast to TD3)
qlearning1 = coax.td_learning.SoftClippedDoubleQLearning(
    q1, pi_targ_list=[pi], q_targ_list=[q1_targ, q2_targ],
    loss_function=coax.value_losses.mse, optimizer=optax.adam(1e-3),
    policy_regularizer=policy_regularizer)
qlearning2 = coax.td_learning.SoftClippedDoubleQLearning(
    q2, pi_targ_list=[pi], q_targ_list=[q1_targ, q2_targ],
    loss_function=coax.value_losses.mse, optimizer=optax.adam(1e-3),
    policy_regularizer=policy_regularizer)
soft_pg = coax.policy_objectives.SoftPG(pi, [q1_targ, q2_targ], optimizer=optax.adam(
    1e-4), regularizer=policy_regularizer)


# train
while env.T < 1000000:
    s, info = env.reset()

    for t in range(env.spec.max_episode_steps):
        a = pi(s)
        s_next, r, done, truncated, info = env.step(a)

        # trace rewards and add transition to replay buffer
        tracer.add(s, a, r, done)
        while tracer:
            buffer.add(tracer.pop())

        # learn
        if len(buffer) >= 5000:
            transition_batch = buffer.sample(batch_size=256)

            # init metrics dict
            metrics = {}

            # flip a coin to decide which of the q-functions to update
            qlearning = qlearning1 if jax.random.bernoulli(q1.rng) else qlearning2
            metrics.update(qlearning.update(transition_batch))

            # delayed policy updates
            if env.T >= 7500 and env.T % 4 == 0:
                metrics.update(soft_pg.update(transition_batch))

            env.record_metrics(metrics)

            # sync target networks
            q1_targ.soft_update(q1, tau=0.005)
            q2_targ.soft_update(q2, tau=0.005)

        if done or truncated:
            break

        s = s_next

    # generate an animated GIF to see what's going on
    if env.period(name='generate_gif', T_period=10000) and env.T > 5000:
        T = env.T - env.T % 10000  # round to 10000s
        coax.utils.generate_gif(
            env=env, policy=pi, filepath=f"./data/gifs/{name}/T{T:08d}.gif")
